# Cell Me

Raport zawiera podsumowanie projektu "Cell Me". Jego celem jest predykcja oceny telefonu komórkowego bazując na recenzji napisanej przez użytkownika danego modelu. Model predykcji wykonany został z użyciem Pythona w oparciu o bibliotekę _scikit-learn_. Wyjście programu stanowi plik .csv zawierający ID recenzji oraz przewidzianą ocenę.


### Zaimportowanie bibliotek
Przed rozpoczęciem prac nad projektem należy załadować niezbędne biblioteki, między innymi _pandas_, _nltk_ oraz _scikit-learn_. 

In [114]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import csv as csv

### Załadowanie oraz obróbka danych
Do zmiennych train_df oraz test_df zostały zapisane odpowiednio treningowy oraz testowy zbiór danych.

In [4]:
train_df = pd.read_csv("train.csv", sep=",")
test_df = pd.read_csv('test.csv', sep=",")

Aby poznać rozmiary powyższych zbiorów danych użyto atrybutu shape, który jest krótką z informacją o ilości wierszy i kolumn.

In [3]:
print('Rozmiar treningowego zbioru danych: ', train_df.shape) 
print('Rozmiar testowego zbioru danych: ', test_df.shape) 

Rozmiar treningowego zbioru danych:  (202645, 6)
Rozmiar testowego zbioru danych:  (211195, 5)


Za pomocą funkcji head można dowiedzieć się jakie zmienne zawiera analizowany data frame.

In [4]:
train_df.head()

,Id,Product Name,Brand Name,Price,Reviews,Rating
0,1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,I feel so LUCKY to have found this used (phone...,5
1,2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,"nice phone, nice up grade from my pantach revu...",4
2,3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,Very pleased,5
3,4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,It works good but it goes slow sometimes but i...,4
4,5,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,Great phone to replace my lost phone. The only...,4


In [5]:
test_df.head()

,Id,Product Name,Brand Name,Price,Reviews
0,202646,Huawei P8 Lite ALE-L23 Factory Unlocked 16GB L...,Huawei,184.9,Excelent
1,202647,Huawei P8 Lite ALE-L23 Factory Unlocked 16GB L...,Huawei,184.9,Fit perfectly. Excelent.
2,202648,Huawei P8 Lite ALE-L23 Factory Unlocked 16GB L...,Huawei,184.9,"It has a very good relation, price/performance."
3,202649,Huawei P8 Lite ALE-L23 Factory Unlocked 16GB L...,Huawei,184.9,Like it a lot
4,202650,Huawei P8 Lite ALE-L23 Factory Unlocked 16GB L...,Huawei,184.9,"Dear , i had a problem wich the wifi senial !!..."


Ze zbioru danych należy wyodrębnić kolumny niezbędne do przeprowadzenia klasyfikacji, czyli te z treścią recenzji oraz oceną. W tym celu utworzono odpowiednio zmienne __review_train__, __raiting_train__ oraz __review_test__

In [128]:
review_train= train_df['Reviews'].astype('str')  
raiting_train = train_df['Rating']
review_test = test_df['Reviews'].astype('str')  

### Przetwarzanie tekstu
Przed przystąpieniem do klasyfikacji, każdą recenzję należy najpierw przekonwertować na macierz. Jej wartości będą reprezentować liczbę wystąpień danego słowa w każdej z recenzji. Aby tego dokonać zastosowano moduł _CountVectorizer_ z biblioteki _Scikit-learn_. Ze zbioru słów wykluczono te, które zawierają liczby oraz słowa ze zbioru _stopwords_

In [129]:
vectorizer = CountVectorizer(analyzer = 'word', stop_words='english',token_pattern=r'\b[^\d\W_]+\b').fit(review_train, review_test) 

In [130]:
review_train = vectorizer.transform(review_train) 
review_test = vectorizer.transform(review_test) 

### Tworzenie modelu klasyfikacji
Biblioteka sci-kit umożliwia proste tworzenie modelu klasyfikacji. W tym przypadku użyty został Wielomianowy naiwny klasyfikator Bayes'a. Jest to jedna z najbardziej efektywnych metod uczenia maszynowego. Służy do klasyfikacji dokumentów tekstowych, ma szerokie zastosowanie w filtrowaniu spamu i w analizie opinii (w analizach statystycznych dla mediów społecznościowych, by zidentyfikować pozytywne i negatywne odczucia konsumentów). Wydaje się więc być idealnym modelem klasyfikacji dla rozpatrywanego problemu.

In [131]:
nb = MultinomialNB()
nb.fit(review_train, raiting_train)
preds = nb.predict(review_test)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Zapis wyniku do pliku _csv._
Wynik predykcji został zapisany do pliku za pomocą biblioteki csv, zgodnie z zadanym formatem.

In [159]:
ids = test_df['Id'].values
predictions_file = open("predict.csv", "w", newline="")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["Id", "Rating"])
result = zip(test_df['Id'], preds)
for el1, el2 in result:
    open_file_object.writerow([el1, el2])
predictions_file.close()